# Data Centre Modelling
## Ryan Jenkinson

### Imports

In [1]:
from dataclasses import dataclass
from typing import Literal

import numpy as np
import pandas as pd
import plotly.express as px
import pypsa
import xarray as xr

In [2]:
np.random.seed(42)  # for reproducibility

### Constants

In [3]:
@dataclass
class Costs:
    capital_cost: float # per MW per year
    marginal_cost: float # per MWh

# Maximum capacity assumptions

In [4]:
# Set to None to ensure the model is just cost optimal without any maximum capacities
# Have defaulted to reasonable maximum capacities based on GB build out, but model chooses much less than this depending on DC size
WIND_MAX_CAPACITY_MW = 1400
SOLAR_MAX_CAPACITY_MW = 600
GAS_MAX_CAPACITY_MW = None
BESS_MAX_CAPACITY_MW, BESS_MAX_CAPACITY_MWH = 300, 600

### Cost Assumptions

In [5]:
## TODO: Sensitivity analysis for costs
cost_scenario: Literal["low", "medium", "high"] = "medium"

## Wind costs
# https://www.gov.uk/government/publications/electricity-generation-costs-2023
# Using 2025 costs, Offshore Wind reference model
KW_TO_MW = 1000
wind_pre_development_costs_gbp_per_mw = {
    "low": 60 * KW_TO_MW,
    "medium": 130 * KW_TO_MW,
    "high": 200 * KW_TO_MW,
}
wind_construction_costs_gbp_per_mw = {
    "low": 1300 * KW_TO_MW,
    "medium": 1500 * KW_TO_MW,
    "high": 2000 * KW_TO_MW,
}
wind_infrastructure_costs_gbp = {
    "low": 56_000_000,
    "medium": 64_300_000,
    "high": 74_100_000,
}
wind_insurance = 3000  # £ per MW per year (insurance cost)

wind_capex_per_mw = (
    wind_pre_development_costs_gbp_per_mw[cost_scenario]
    + wind_construction_costs_gbp_per_mw[cost_scenario]
    + wind_insurance
)  # £ per MW
wind_lifetime_years = 30
wind_discount_rate = 0.07
annuitised_wind_cost = (wind_capex_per_mw * wind_discount_rate) / (
    1 - (1 + wind_discount_rate) ** -wind_lifetime_years
)
wind_fixed_opex = 43300  # £ per MW per year (fixed O&M)
wind_variable_operation_and_management_per_mwh = 1  # £ per MWh_elec (non-fuel variable O&M)

wind_connection_and_use_of_system_charges = 44800  # £ per MW per year (grid connection costs) [NOTE: Not used in model as a microgrid]
WIND_COSTS = Costs(
    capital_cost=annuitised_wind_cost + wind_fixed_opex,  # £ per MW per year
    marginal_cost=wind_variable_operation_and_management_per_mwh,  # £ per MWh
)

## Solar costs
# https://www.gov.uk/government/publications/electricity-generation-costs-2023
# Using 2025 costs, "Large-scale Solar" reference model
KW_TO_MW = 1000
solar_pre_development_costs_gbp_per_mw = {
    "low": 10 * KW_TO_MW,
    "medium": 50 * KW_TO_MW,
    "high": 110 * KW_TO_MW,
}
solar_construction_costs_gbp_per_mw = {
    "low": 300 * KW_TO_MW,
    "medium": 400 * KW_TO_MW,
    "high": 400 * KW_TO_MW,
}
solar_infrastructure_costs_gbp = {
    "low": 1_300_000,
    "medium": 1_400_000,
    "high": 1_500_000,
}
solar_insurance = 2000  # £ per MW per year (insurance cost)

solar_capex_per_mw = (
    solar_pre_development_costs_gbp_per_mw[cost_scenario]
    + solar_construction_costs_gbp_per_mw[cost_scenario]
    + solar_insurance
)  # £ per MW
solar_lifetime_years = 35
solar_discount_rate = 0.07
annuitised_solar_cost = (solar_capex_per_mw * solar_discount_rate) / (
    1 - (1 + solar_discount_rate) ** -solar_lifetime_years
)
solar_fixed_opex = 6000  # £ per MW per year (fixed O&M)
solar_variable_operation_and_management_mwh = 0  # £ per MWh_elec (non-fuel variable O&M)

solar_connection_and_use_of_system_charges = 1300  # £ per MW per year (grid connection costs) [NOTE: Not used in model as a microgrid]
SOLAR_COSTS = Costs(
    capital_cost=annuitised_solar_cost + solar_fixed_opex,  # £ per MW per year
    marginal_cost=solar_variable_operation_and_management_mwh,  # £ per MWh
)

## Gas costs
# Assume OCGT (Open Cycle Gas Turbine) as the reference model
# CCGTs are an alternative for continuous, baseload power generation but they have a much longer start-up time and a significantly larger and more complex footprint so we use OCGT costs here
# Operating Hours: 500 hr vs. 2000 hr --> 500hr is more akin to "backup" generator. 2000hr is equivalent to a standard 8 hour workday every weekday. So we use the 500hr variant
# OCGT 100MW 500 hr -> Logical starting point. Most data centres use 10 x 10MW backups so modelling as a 100MW is a good proxy
# OCGT 299MW 500 hr --> common to model even larger data centres as after 300MW there is much more complicated regulations
# Chosen reference model: OCGT 500hr 100MW
# To use a different reference model, see:
# https://www.gov.uk/government/publications/electricity-generation-costs-2023

gas_reference_model: Literal["100MW", "299MW"] = (
    "100MW"  # Reference model for gas turbine costs
)

gas_fuel_efficiency = {
    "100MW": 0.35,  # Efficiency of the 100MW OCGT (MWh_elec / MWh_fuel)
    "299MW": 0.34,  # Efficiency of the 299MW OCGT (MWh_elec / MWh_fuel)
}
gas_pre_development_costs_gbp_per_mw = {
    "100MW": {"low": 80 * KW_TO_MW, "medium": 90 * KW_TO_MW, "high": 110 * KW_TO_MW},
    "299MW": {"low": 30 * KW_TO_MW, "medium": 40 * KW_TO_MW, "high": 40 * KW_TO_MW},
}
gas_construction_costs_gbp_per_mw = {
    "100MW": {"low": 600 * KW_TO_MW, "medium": 700 * KW_TO_MW, "high": 900 * KW_TO_MW},
    "299MW": {"low": 300 * KW_TO_MW, "medium": 500 * KW_TO_MW, "high": 800 * KW_TO_MW},
}
gas_infrastructure_costs_gbp = {
    "100MW": {"low": 7_200_000, "medium": 14_400_000, "high": 28_700_000},
    "299MW": {"low": 7_800_000, "medium": 15_500_000, "high": 31_000_000},
}  # TODO: How to add this into the model? Do we annuitise it? But the other costs are annuitised per MW... :thinking:
gas_insurance = {
    "100MW": 2900,  # £ per MW per year (insurance cost for 100MW OCGT)
    "299MW": 1800,  # £ per MW per year (insurance cost for 299MW OCGT)
}
gas_connection_use_of_system_charges = 2700  # NOTE: Not used in model as a microgrid

gas_capex_per_mw = (
    gas_pre_development_costs_gbp_per_mw[gas_reference_model][cost_scenario]
    + gas_construction_costs_gbp_per_mw[gas_reference_model][cost_scenario]
    + gas_insurance[gas_reference_model]
)
gas_lifetime_years = 25
gas_discount_rate = 0.07
annuitised_gas_cost = (gas_capex_per_mw * gas_discount_rate) / (
    1 - (1 + gas_discount_rate) ** -gas_lifetime_years
)
gas_fixed_opex = {
    "100MW": 11300,  # £ per MW per year (fixed O&M for 100MW OCGT)
    "299MW": 7300,  # £ per MW per year (fixed O&M for 299MW OCGT)
}[gas_reference_model]  # Fixed O&M cost based on the reference model
gas_fuel_price_pence_per_therm = 80 # Taken from GB Sep 2025 Gas Price
PENCE_PER_THERM_TO_GBP_PER_MWH = 2.93071
gas_fuel_price_mwh = gas_fuel_price_pence_per_therm / PENCE_PER_THERM_TO_GBP_PER_MWH
gas_variable_operation_and_management_mwh = {
    "100MW": 2,  # £ per MWh_elec (non-fuel variable O&M for 100MW OCGT)
    "299MW": 1,  # £ per MWh_elec (non-fuel variable O&M for 299MW OCGT)
}[gas_reference_model]
gas_marginal_cost_fuel = (
    gas_fuel_price_mwh / gas_fuel_efficiency[gas_reference_model]
)  # £ per MWh_elec
GAS_COSTS = Costs(
    capital_cost=annuitised_gas_cost + gas_fixed_opex,  # £ per MW per year
    marginal_cost=gas_marginal_cost_fuel
    + gas_variable_operation_and_management_mwh,  # £ per MWh_elec
)

## BESS (Battery Energy Storage System) costs
# https://www.utilitydive.com/news/tariffs-to-spike-power-generation-costs-reports/750133/
# https://www.numberanalytics.com/blog/energy-storage-economics
bess_energy_capex_per_mwh = {
    "low": 140_000,  # £ per MWh
    "medium": 170_000,  # £ per MWh
    "high": 200_000,  # £ per MWh
}
bess_power_capex_per_mw = {
    "low": 450_000,  # £ per MW
    "medium": 540_000,  # £ per MW
    "high": 630_000,  # £ per MW
}
bess_lifetime_years = 15
bess_discount_rate = 0.07

annuitised_bess_energy_cost = (
    bess_energy_capex_per_mwh[cost_scenario] * bess_discount_rate
) / (1 - (1 + bess_discount_rate) ** -bess_lifetime_years)
bess_capital_cost_energy = (
    annuitised_bess_energy_cost  # Cost for energy capacity (Store)
)

annuitised_bess_power_cost = (
    bess_power_capex_per_mw[cost_scenario] * bess_discount_rate
) / (1 - (1 + bess_discount_rate) ** -bess_lifetime_years)
bess_fixed_opex_per_mw_power = 20000  # Fixed OPEX for power conversion part
bess_capital_cost_power = (
    annuitised_bess_power_cost + bess_fixed_opex_per_mw_power
)  # Cost for power capacity (Links)

bess_round_trip_efficiency = 0.87
standing_loss_per_hour = 0.001
bess_marginal_cost_links = 0.001  # Marginal cost for charging/discharging

BESS_COSTS = Costs(
    capital_cost=bess_capital_cost_energy,  # £ per MWh per year
    marginal_cost=0,
)
BESS_LINK_COSTS = Costs(
    capital_cost=bess_capital_cost_power,  # £ per MW per year
    marginal_cost=bess_marginal_cost_links,  # £ per MWh_elec
)

# Cannot add in infrastructure costs to calculations above as they are just £ total not depending on load e.g per MW
TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS = (
    (wind_infrastructure_costs_gbp[cost_scenario] * wind_discount_rate) / (1 - (1 + wind_discount_rate) ** -wind_lifetime_years)
    + ((gas_infrastructure_costs_gbp[gas_reference_model][cost_scenario] * gas_discount_rate) / (1 - (1 + gas_discount_rate) ** -gas_lifetime_years))
)
TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_WITH_SOLAR = TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS + (
    (solar_infrastructure_costs_gbp[cost_scenario] * solar_discount_rate) / (1 - (1 + solar_discount_rate) ** -solar_lifetime_years)
)

# Small Modular Reactor costs
SMR_COST_FOR_450MW_UNIT = 2.5e9 # £ per unit, assuming a single unit
smr_lifetime_years = 60  # Assumed lifetime of the SMR in years, from Rolls Royce : https://gda.rolls-royce-smr.com/our-technology#:~:text=Over%20its%2060%2Dyear%20lifetime,type%20of%20waste%20is%20it%3F
smr_discount_rate = 0.07  # assumed the same as wind and gas
smr_capital_cost = (
    (SMR_COST_FOR_450MW_UNIT / 450) * smr_discount_rate / (1 - (1 + smr_discount_rate) ** -smr_lifetime_years)
)  # Annuitised capital

SMR_COSTS = Costs(
    capital_cost=smr_capital_cost,
    marginal_cost=50,  # £ per MWh_elec, assumed operational cost, conservative value from research
)

smr_infrastructure_costs_gbp = 0 # TODO: Get any other external infrastructure costs
TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_SMR = (
    smr_infrastructure_costs_gbp * smr_discount_rate
) / (1 - (1 + smr_discount_rate) ** -smr_lifetime_years)

# TODO: Consolidate the costs above, but this is fine for now

In [6]:
DATA_CENTRE_LOAD_MW = 115  # MW, constant demand including PUE

## Wind and Solar Profiles

Get the availabilities of wind and solar profiles

In [7]:
REFERENCE_YEAR: Literal["2019", "2023"] = (
    "2023"  # Reference year for the model, using most recent available
)
# Wind Data
wind_data = xr.load_dataset(f"../data/profile_39_offshorewind-{REFERENCE_YEAR}.nc")
wind_availability_df = wind_data["profile"].to_dataframe().reset_index()
wind_availability_pu = wind_availability_df[wind_availability_df["bus"] == "GB2 0"].set_index("time")[
    "profile"
].clip(0, 1).reset_index(drop=True)
wind_capacity_factor = (
    0.61  # Target capacity factor for the wind farm (Source: DESNZ, similar to actuals)
)
wind_availability_pu = wind_capacity_factor * (
    wind_availability_pu / wind_availability_pu.mean()
)
wind_availability_pu = wind_availability_pu.to_numpy()

# Solar Data
solar_profile = xr.load_dataset(f"../data/profile_39_solar-{REFERENCE_YEAR}.nc")
solar_availability_df = solar_profile["profile"].to_dataframe().reset_index()
solar_availability_pu = solar_availability_df[solar_availability_df.bus == "GB2 0"].set_index("time")[
    "profile"
].clip(0, 1).reset_index(drop=True)
solar_capacity_factor = (
    0.11  # Target capacity factor for the solar (Source: DESNZ, similar to actuals)
)
solar_availability_pu = solar_capacity_factor * (
    solar_availability_pu / solar_availability_pu.mean()
)
solar_availability_pu = solar_availability_pu.to_numpy()

# Plot rolling hour availability of wind and solar on the same chart
availability_df = pd.DataFrame(
    {
        "Wind Availability (pu)": wind_availability_pu,
        "Solar Availability (pu)": solar_availability_pu,
    }
)
# plot the target capacity factors as horizontal lines
fig = px.line(
    availability_df.rolling(window=24).mean(),
    title="Rolling 24 Hour Average Availability of Wind and Solar",
    labels={"index": "Hour", "value": "Availability (pu)"},
).add_hline(
    y=wind_capacity_factor,
    line_dash="dash",
    line_color="lightgreen",
    annotation_text="Target Wind Capacity Factor",
    annotation_position="top left",
).add_hline(
    y=solar_capacity_factor,
    line_dash="dash",
    line_color="gold",
    annotation_text="Target Solar Capacity Factor",
    annotation_position="top left",
)
fig.show()

### Model

In [8]:
MODEL_TIME_FREQUENCY = pd.Timedelta("1h")  # Time frequency for the model
GENERATION_TYPES: Literal["wind", "solar", "gas", "bess", "smr"] = (
    "offshore_wind",
    "gas",
    "bess",
)
def create_model(data_centre_load_mw: float, generation_types=GENERATION_TYPES, **solar_kwargs: dict):
    # Define simulation period: a full year, hourly
    snapshots = pd.date_range(
        start=f"{REFERENCE_YEAR}-01-01 00:00", end=f"{REFERENCE_YEAR}-12-31 23:00", freq=MODEL_TIME_FREQUENCY
    )
    n = pypsa.Network(snapshots=snapshots)

    # --- DEFINE CARRIERS ---
    n.add("Carrier", "AC")  # For AC buses
    n.add("Carrier", "gas", co2_emissions=0.185)  # tCO2/MWh_th, typical for natural gas
    n.add("Carrier", "offshore_wind")
    n.add("Carrier", "battery")  # Carrier for battery components

    # --- DEFINE BUSES ---
    n.add("Bus", "electricity_bus", carrier="AC")
    if "bess" in generation_types:
        n.add("Bus", "bess_connection_bus", carrier="AC")

    # --- DEFINE LOAD ---
    # Load (Data Centre): The data centre is modelled as a load with a constant power demand.
    constant_demand_profile = pd.Series(data_centre_load_mw, index=n.snapshots)
    n.add("Load", "data_centre_load", bus="electricity_bus", p_set=constant_demand_profile)

    # --- DEFINE GENERATORS ---
    # Generator (Offshore Wind Farm): The offshore wind farm's capacity is extendable.
    if "offshore_wind" in generation_types:
        n.add(
            "Generator",
            "offshore_wind_farm",
            bus="electricity_bus",
            carrier="offshore_wind",
            p_nom_extendable=True,  # Allow capacity to be extended. To cover what we need to for the data centre load
            capital_cost=WIND_COSTS.capital_cost,
            marginal_cost=WIND_COSTS.marginal_cost,
            p_max_pu=pd.Series(wind_availability_pu, index=n.snapshots),
            p_nom_max=WIND_MAX_CAPACITY_MW
        )

    if "solar" in generation_types:
        n.add(
            "Generator",
            "solar",
            bus="electricity_bus",
            carrier="AC",
            p_nom_extendable=True,  # Allow capacity to be extended to cover the data centre load
            committable=False,
            capital_cost=SOLAR_COSTS.capital_cost,
            marginal_cost=SOLAR_COSTS.marginal_cost,
            p_max_pu=pd.Series(solar_availability_pu, index=n.snapshots),
            p_nom_max=SOLAR_MAX_CAPACITY_MW,
            **solar_kwargs
        )

    # Generator (Gas Turbine Backup): The gas turbine provides backup, and its capacity is also extendable.
    if "gas" in generation_types:
        n.add(
            "Generator",
            "gas_turbine",
            bus="electricity_bus",
            carrier="gas",
            p_nom_extendable=True,  # Allow capacity to be extended to cover the data centre load
            committable=False,
            capital_cost=GAS_COSTS.capital_cost,
            marginal_cost=GAS_COSTS.marginal_cost,
            p_nom_max=GAS_MAX_CAPACITY_MW
        )

    # --- DEFINE STORAGE (BESS) ---
    if "bess" in generation_types:
        # Store component representing the energy capacity of the BESS, and each of the links for charging and discharging.
        n.add(
            "Store",
            "bess_storage",
            bus="bess_connection_bus",
            carrier="battery",
            e_nom_extendable=True,
            e_cyclic=True,
            standing_loss=standing_loss_per_hour,
            capital_cost=BESS_COSTS.capital_cost,
            e_nom_max=BESS_MAX_CAPACITY_MWH,
        )
        n.add(
            "Link",
            "bess_charger",
            bus0="electricity_bus",
            bus1="bess_connection_bus",
            carrier="battery",
            p_nom_extendable=True,
            capital_cost=BESS_LINK_COSTS.capital_cost,
            efficiency=bess_round_trip_efficiency**0.5,
            marginal_cost=BESS_LINK_COSTS.marginal_cost,
            p_nom_max=BESS_MAX_CAPACITY_MW
        )
        n.add(
            "Link",
            "bess_discharger",
            bus0="bess_connection_bus",
            bus1="electricity_bus",
            carrier="battery",
            p_nom_extendable=True,
            capital_cost=0,  # Assuming power CAPEX is fully on charger link, or split if preferred
            efficiency=bess_round_trip_efficiency**0.5,
            marginal_cost=BESS_LINK_COSTS.marginal_cost,
            p_nom_max=BESS_MAX_CAPACITY_MW
        )

    if "smr" in generation_types:
        n.add(
            "Generator",
            "smr",
            bus="electricity_bus",
            carrier="AC",
            p_nom_extendable=True,
            committable=False,
            capital_cost=SMR_COSTS.capital_cost,
            marginal_cost=SMR_COSTS.marginal_cost,
        )

    # --- CONSISTENCY CHECK & SOLVE ---

    # Perform consistency check
    try:
        n.consistency_check()
        print("Network consistency check passed before solving.")
    except Exception as e:
        print(f"Consistency check error before solving: {e}")

    return n

In [9]:
# Ensure you have a solver installed (e.g., cbc, glpk, gurobi)
SOLVER_NAME = "highs"  # or 'glpk', 'gurobi', etc.


def solve_model(n: pypsa.Network, solver_name: str = SOLVER_NAME, print_results: bool = True):
    """Solve the PyPSA network model using the specified solver.

    Parameters:
    - n: PyPSA Network object
    - solver_name: Name of the solver to use (e.g., 'cbc', 'glpk')
    - print_results: Whether to print the results after solving
    """
    try:
        # Solve the model using the optimize interface, common with linopy backend
        # The `solve_model` method handles the optimization.
        n.optimize(solver_name=solver_name)
        if not print_results:
            return n
        print("Model solved successfully.")

        # --- DISPLAY RESULTS ---
        print("\n--- Optimization Results ---")
        # Objective value is typically stored in n.model.objective after solving with n.optimize
        obj_value = n.model.objective.value
        print(f"Objective Value: {obj_value:.2f} £ (Total Cost)")

        print("\nOptimal Capacities:")
        # Check if components exist in results before trying to access them
        # Optimal capacities are in p_nom_opt (for Generators, Links) and e_nom_opt (for Stores)
        if (
            "offshore_wind_farm" in n.generators.index
            and "p_nom_opt" in n.generators.columns
        ):
            print(
                f"  Offshore Wind Farm (p_nom_opt): {n.generators.loc['offshore_wind_farm', 'p_nom_opt']:.2f} MW"
            )
        if "solar" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  Solar (p_nom_opt): {n.generators.loc['solar', 'p_nom_opt']:.2f} MW"
            )
        if "gas_turbine" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  Gas Turbine (p_nom_opt): {n.generators.loc['gas_turbine', 'p_nom_opt']:.2f} MW"
            )
        if "bess_storage" in n.stores.index and "e_nom_opt" in n.stores.columns:
            print(
                f"  BESS Storage Energy Capacity (e_nom_opt): {n.stores.loc['bess_storage', 'e_nom_opt']:.2f} MWh"
            )
        if "bess_charger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Charger Power Capacity (p_nom_opt): {n.links.loc['bess_charger', 'p_nom_opt']:.2f} MW"
            )
        if "bess_discharger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Discharger Power Capacity (p_nom_opt): {n.links.loc['bess_discharger', 'p_nom_opt']:.2f} MW"
            )
        if "smr" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  SMR (p_nom_opt): {n.generators.loc['smr', 'p_nom_opt']:.2f} MW"
            )

    except Exception as e:
        print(f"An error occurred during model solution or results processing: {e}")
        print("Please ensure you have a compatible solver installed and in your PATH.")
        print("Common solvers: cbc, highs. You might need to install them.")
        print(
            "Also ensure Linopy (PyPSA's modeling backend) is installed and functioning correctly."
        )
    return n

In [10]:
def get_optimal_capacities(n: pypsa.Network):
    """Get the optimal capacities of the network components after solving."""
    optimal_capacities = {
        "offshore_wind_farm": n.generators.loc["offshore_wind_farm", "p_nom_opt"],
        "gas_turbine": n.generators.loc["gas_turbine", "p_nom_opt"],
        "bess_storage": n.stores.loc["bess_storage", "e_nom_opt"],
        "bess_charger": n.links.loc["bess_charger", "p_nom_opt"],
        "bess_discharger": n.links.loc["bess_discharger", "p_nom_opt"],
    }
    return optimal_capacities


def get_asset_expenditure(n: pypsa.Network):
    stats = n.statistics()
    return {
        "gas": (
            stats.loc[("Generator", "gas"), "Capital Expenditure"]
            + stats.loc[("Generator", "gas"), "Operational Expenditure"]
        ),
        "offshore_wind": (
            stats.loc[("Generator", "offshore_wind"), "Capital Expenditure"]
            + stats.loc[("Generator", "offshore_wind"), "Operational Expenditure"]
        ),
        "battery": (
            stats.loc[("Link", "battery"), "Capital Expenditure"]
            + stats.loc[("Link", "battery"), "Operational Expenditure"]
            + stats.loc[("Store", "battery"), "Capital Expenditure"]
            + stats.loc[("Store", "battery"), "Operational Expenditure"]
        ),
    }

# Run modelling scenarios

In [11]:
scenarios = {
    "smr_only": ("smr"),
    "wind_bess_gas": ("offshore_wind", "bess", "gas"),
    "wind_bess_gas_solar": ("offshore_wind", "bess", "gas", "solar"),
}

scenario_networks = {}
for scenario_name, generation_types in scenarios.items():
    n = create_model(data_centre_load_mw=DATA_CENTRE_LOAD_MW, generation_types=generation_types)
    n = solve_model(n, print_results=True)
    scenario_networks[scenario_name] = n

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.04s


Network consistency check passed before solving.
Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-tp8sc6o0 has 26281 rows; 8761 cols; 35041 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e+01, 4e+05]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+02]
Presolving model
8760 rows, 1 cols, 8760 nonzeros  0s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 8761 primals, 26281 duals
Objective: 9.59e+07
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper were not assigned to the network.
/Users/ryan.jenkinson1/repos/modelling/data-centre-modelling/.venv/lib/python3.11/site-packages/pypsa/network/io.py:1700: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-26281); columns 0(-8761); elements 0(-35041) - Reduced to empty
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-tp8sc6o0
Model status        : Optimal
Objective value     :  9.5877560736e+07
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.01
Writing the solution to /private/var/folders/r9/dprdmb3d7gs81f5ggtwv0lyw0000gn/T/linopy-solve-x6ctksny.sol
Model solved successfully.

--- Optimization Results ---
Objective Value: 95877560.74 £ (Total Cost)

Optimal Capacities:
  SMR (p_nom_opt): 115.00 MW
Network consistency check passed before solving.


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 233.63it/s]
INFO:linopy.io: Writing time: 0.24s


...

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-k6m8tbrd has 131411 rows; 61326 cols; 249206 nonzeros
Coefficient ranges:
  Matrix [1e-02, 1e+00]
  Cost   [1e-03, 2e+05]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+03]
Presolving model
65235 rows, 56480 cols, 169425 nonzeros  0s
Dependent equations search running on 17520 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
56475 rows, 47720 cols, 151905 nonzeros  0s
Presolve : Reductions: rows 56475(-74936); columns 47720(-13606); elements 151905(-97301)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 8760(1.0074e+06) 0s
      40551     5.2788174934e+07 Pr: 11574(327591) 5s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 61326 primals, 131411 duals
Objective: 5.62e+07
Solver model: available
Solver message: Optimal



      53429     5.6232152715e+07 Pr: 0(0) 6s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-k6m8tbrd
Model status        : Optimal
Simplex   iterations: 53429
Objective value     :  5.6232152715e+07
P-D objective error :  1.1262228361e-14
HiGHS run time      :          6.26
Writing the solution to /private/var/folders/r9/dprdmb3d7gs81f5ggtwv0lyw0000gn/T/linopy-solve-8b7c42qi.sol


INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


Model solved successfully.

--- Optimization Results ---
Objective Value: 56232152.71 £ (Total Cost)

Optimal Capacities:
  Offshore Wind Farm (p_nom_opt): 144.80 MW
  Solar (p_nom_opt): 87.02 MW
  Gas Turbine (p_nom_opt): 109.56 MW
  BESS Storage Energy Capacity (e_nom_opt): 4.93 MWh
  BESS Charger Power Capacity (p_nom_opt): 0.22 MW
  BESS Discharger Power Capacity (p_nom_opt): 300.00 MW


In [12]:
SCENARIO_TO_PLOT = "wind_bess_gas" #"wind_bess_gas"
GENERATOR_NAMES_READABLE = {
    "smr": "SMR",
    "bess": "BESS",
    "offshore_wind_farm": "Offshore Wind",
    "solar": "Solar",
    "gas_turbine": "Gas Turbine",
}
COLOUR_MAP = {
    "SMR": "purple",
    "BESS": "orange",
    "Offshore Wind": "lightgreen",
    "Solar": "gold",
    "Gas Turbine": "darkgrey"
}

n = scenario_networks[SCENARIO_TO_PLOT]

generators = n.generators.index.tolist()

generator_power = n.generators_t.p
data_centre_power = n.loads_t.p["data_centre_load"]
asset_power = generator_power.copy()
asset_power["dc_load"] = data_centre_power
try:
    battery_power = n.links_t.p1["bess_charger"] - n.links_t.p1["bess_discharger"]
    asset_power["bess"] = battery_power
    generators += ["bess"]
except:
    pass


asset_energy_mwh_per_day = asset_power.groupby(pd.Grouper(freq="D")).apply(lambda x: x.abs().sum().mul(
    MODEL_TIME_FREQUENCY.total_seconds() / 3600
))  # Convert power to energy (MWh)
asset_energy_mwh_per_month = asset_power.groupby(pd.Grouper(freq="M")).apply(lambda x: x.abs().sum().mul(
    MODEL_TIME_FREQUENCY.total_seconds() / 3600
))  # Convert power to energy (MWh)

fig = px.line(
    data_frame=asset_power.rename(columns=GENERATOR_NAMES_READABLE),
    title="Asset Power Output Over Time",
    labels={"value": "Power (MW)", "variable": "Asset"},
    template="plotly_white",
    color_discrete_map=COLOUR_MAP
)
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=asset_power_output_over_time.html")
fig.show()

# For each time period in the day, create a stacked bar chart showing the amount of energy used from each of the generators to meet the data centre load
asset_energy_mwh_per_day = asset_energy_mwh_per_day.rename(columns=GENERATOR_NAMES_READABLE)
fig = px.bar(
    data_frame=asset_energy_mwh_per_day,
    x=asset_energy_mwh_per_day.index,
    y=list(set(asset_energy_mwh_per_day.columns.to_list()) - {"dc_load"}),
    title="Energy Used from Each Generator to Meet Data Centre Load on Each Day of the Year",
    labels={"value": "Power (MW)", "variable": "Generator"},
    template="plotly_white",
    color_discrete_map=COLOUR_MAP
)
# Add a line for the data centre load
fig.add_scatter(
    x=asset_energy_mwh_per_day.index,
    y=asset_energy_mwh_per_day["dc_load"],
    mode="lines",
    name="Data Centre Load",
    line=dict(color="black", width=2),
)
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=energy_used_per_day.html")
fig.show()

fig.update_xaxes(range=["2023-08-14", "2023-09-24"])
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=energy_used_per_day_zoomed.html")

# Data used monthly
# Add percentage columns for each generator
for col in generators:
    asset_energy_mwh_per_month[f"{col}_pct"] = (
        asset_energy_mwh_per_month[col] / asset_energy_mwh_per_month["dc_load"] * 100
    ).round(1).astype(str) + "%"

# Melt the DataFrame for easier plotting with text labels
melted = asset_energy_mwh_per_month.reset_index().melt(
    id_vars=["snapshot"],
    value_vars=generators,
    var_name="Generator",
    value_name="Energy (MWh)",
)
# Add percentage labels
melted["Percentage"] = [
    asset_energy_mwh_per_month.loc[row["snapshot"], f"{row['Generator']}_pct"]
    for _, row in melted.iterrows()
]
melted["Generator"] = melted["Generator"].replace(GENERATOR_NAMES_READABLE)

fig = px.bar(
    melted,
    x="snapshot",
    y="Energy (MWh)",
    color="Generator",
    title="Energy Used from Each Generator to Meet Data Centre Load on Each Month of the Year",
    labels={"snapshot": "Month"},
    template="plotly_white",
    color_discrete_map=COLOUR_MAP,
    text="Percentage"
)
# Add a line for the data centre load
fig.add_scatter(
    x=asset_energy_mwh_per_month.index,
    y=asset_energy_mwh_per_month["dc_load"],
    mode="lines",
    name="Data Centre Load",
    line=dict(color="black", width=2),
)
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=energy_used_per_month.html")

fig.show()

/var/folders/r9/dprdmb3d7gs81f5ggtwv0lyw0000gn/T/ipykernel_82210/3139615772.py:36: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [13]:
# Get optimised powers for each asset
# TODO: Fix ordering of generators
percentages = []
labels = []
if "smr" in generators:
    smr_opt_power = n.generators.loc["smr", "p_nom_opt"]
    smr_pct = smr_opt_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(smr_pct)
    labels.append("<b>[SMR]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(smr_opt_power, smr_pct))
if "offshore_wind_farm" in generators:
    wind_opt_power = n.generators.loc["offshore_wind_farm", "p_nom_opt"]
    wind_pct = wind_opt_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(wind_pct)
    labels.append("<b>[Offshore Wind]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(wind_opt_power, wind_pct))
if "solar" in generators:
    solar_opt_power = n.generators.loc["solar", "p_nom_opt"]
    solar_pct = solar_opt_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(solar_pct)
    labels.append("<b>[Solar]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(solar_opt_power, solar_pct))
if "gas_turbine" in generators:
    gas_opt_power = n.generators.loc["gas_turbine", "p_nom_opt"]
    gas_pct = gas_opt_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(gas_pct)
    labels.append("<b>[Gas Turbine]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(gas_opt_power, gas_pct))
if "bess" in generators:
    bess_opt_energy = n.stores.loc["bess_storage", "e_nom_opt"]
    bess_input_power = n.links.loc["bess_charger", "p_nom_opt"]
    bess_output_power = n.links.loc["bess_discharger", "p_nom_opt"]
    bess_avg_power = (bess_input_power + bess_output_power) / 2
    bess_pct = bess_avg_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(bess_pct)
    labels.append("<b>[BESS]</b><br><i>Avg Power (Import/Export):</i> {:.1f} MW<br><i>Energy:</i> {:.1f} MWh<br><i>({:.1f}%)</i>".format(bess_avg_power, bess_opt_energy, bess_pct))

fig = px.bar(
    x=percentages,
    y=["Data Centre Load"] * len(generators),
    orientation="h",
    text=labels,
    color=generators,
    color_discrete_map={
        "offshore_wind_farm": "lightgreen",
        "gas_turbine": "darkgrey",
        "bess": "orange",
        "smr": "purple",
        "solar": "gold",
    },
    title=f"Optimised Microgrid Generation Powers as Percentage of Data Centre Load<br><i>(Assuming {DATA_CENTRE_LOAD_MW} MW DC Load)</i>",
    labels={"x": "Percentage of DC Load (%)", "y": ""},
)

# Add black outline to each bar
for trace in fig.data:
    trace.marker.line.width = 2
    trace.marker.line.color = "black"

fig.update_traces(texttemplate="%{text}", textposition="inside")
fig.update_layout(
    barmode="stack",
    xaxis_title="Percentage of DC Load (%)",
    yaxis_title="",
    template="plotly_white",
    showlegend=False,
)
fig.show()

In [14]:
total_gas_mwh_usage = n.generators_t.p["gas_turbine"].sum()
total_data_centre_mwh_usage = n.loads_t.p["data_centre_load"].sum()
print(
    f"Ratio of data centre load covered by gas turbine: {total_gas_mwh_usage / total_data_centre_mwh_usage:.2%}"
)

Ratio of data centre load covered by gas turbine: 24.84%


In [15]:
HOURS_PER_YEAR = 8760  # Total hours in a year
NUCLEAR_PPA_COST_PER_MWH = 133  # 92.5 £2012/MWh in todays money
CARBON_FREE_ENERGY_PREMIUM_PERCENTAGE = 0  # Can assume a percentage hyperscalers would be willing to pay e.g 10%. Be conservative.


def calculate_costs(data_centre_load_mw: float):
    print(f"Data Centre Load: {load} MW")
    # Create the models
    smr_network = create_model(
        data_centre_load_mw=data_centre_load_mw, generation_types=["smr"]
    )
    microgrid_network = create_model(
        data_centre_load_mw=data_centre_load_mw,
        generation_types=["gas", "bess", "offshore_wind"],
    )
    microgrid_network_with_solar = create_model(
        data_centre_load_mw=data_centre_load_mw,
        generation_types=["gas", "bess", "offshore_wind", "solar"],
    )
    # Solve the models
    smr_network = solve_model(smr_network, print_results=False)
    microgrid_network = solve_model(microgrid_network, print_results=False)
    microgrid_network_with_solar = solve_model(microgrid_network_with_solar, print_results=False)

    # Calculate the costs
    cost_of_running_microgrid_per_year = (
        microgrid_network.model.objective.value + TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS
    )
    cost_of_running_microgrid_with_solar_per_year = (
        microgrid_network_with_solar.model.objective.value
        + TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_WITH_SOLAR
    )
    # i.e 1 if up to 500MW, 2 if up to 1000 MW etc
    number_of_smrs_needed = (load // 500) + 1
    cost_of_running_with_nuclear_ppa_per_year = (
        load
        * HOURS_PER_YEAR
        * (NUCLEAR_PPA_COST_PER_MWH * (1 + CARBON_FREE_ENERGY_PREMIUM_PERCENTAGE))
    )
    cost_of_running_with_nuclear_smrs_per_year = (
        smr_network.model.objective.value
        + number_of_smrs_needed * TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_SMR
    )  # £ per year, with the SMR being built for the purpose just of the data centre

    print(
        f"Cost of running the microgrid per year: £{cost_of_running_microgrid_per_year:,.2f}"
    )
    print(
        f"Cost of running the microgrid with solar per year: £{cost_of_running_microgrid_with_solar_per_year:,.2f}"
    )
    print(
        f"Cost of running with a purpose-built SMR per year: £{cost_of_running_with_nuclear_smrs_per_year:,.2f}"
    )
    # Calculate the percentage difference in costs
    cost_difference_percentage = (
        (
            cost_of_running_microgrid_per_year
            - cost_of_running_with_nuclear_smrs_per_year
        )
        / cost_of_running_with_nuclear_smrs_per_year
    ) * 100
    cost_difference_percentage_with_solar = (
        (
            cost_of_running_microgrid_with_solar_per_year
            - cost_of_running_with_nuclear_smrs_per_year
        )
        / cost_of_running_with_nuclear_smrs_per_year
    ) * 100
    print(f"Percentage difference in costs: {cost_difference_percentage:.2f}%")
    print(f"Percentage difference in costs (with solar): {cost_difference_percentage_with_solar:.2f}%")
    return (
        cost_difference_percentage,
        cost_difference_percentage_with_solar,
        cost_of_running_with_nuclear_smrs_per_year,
        cost_of_running_microgrid_per_year,
        cost_of_running_microgrid_with_solar_per_year,
    )

cost_difference_percentages = {}
cost_difference_percentages_with_solar = {}
cost_microgrid = {}
cost_microgrid_with_solar = {}
cost_nuclear_smr = {}

DC_LOADS = [50, 100, 200, 300, 400, 500, DATA_CENTRE_LOAD_MW]
DC_LOADS = [DATA_CENTRE_LOAD_MW]  # For just the one load case

for load in DC_LOADS:
    (
        cost_difference_percentage,
        cost_difference_percentage_with_solar,
        cost_of_running_with_nuclear_smrs_per_year,
        cost_of_running_microgrid_per_year,
        cost_of_running_microgrid_with_solar_per_year,
    ) = calculate_costs(load)
    cost_difference_percentages[load] = cost_difference_percentage
    cost_difference_percentages_with_solar[load] = cost_difference_percentage_with_solar
    cost_microgrid[load] = cost_of_running_microgrid_per_year
    cost_microgrid_with_solar[load] = cost_of_running_microgrid_with_solar_per_year
    cost_nuclear_smr[load] = cost_of_running_with_nuclear_smrs_per_year


Data Centre Load: 115 MW
Network consistency check passed before solving.
Network consistency check passed before solving.
Network consistency check passed before solving.


/Users/ryan.jenkinson1/repos/modelling/data-centre-modelling/.venv/lib/python3.11/site-packages/pypsa/network/io.py:1700: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/Users/ryan.jenkinson1/repos/modelling/data-centre-modelling/.venv/lib/python3.11/site-packages/pypsa/network/io.py:1700: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.04s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 8761 primals, 26281 duals
Objective

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-bv3n5jil has 26281 rows; 8761 cols; 35041 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e+01, 4e+05]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+02]
Presolving model
8760 rows, 1 cols, 8760 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-26281); columns 0(-8761); elements 0(-35041) - Reduced to empty
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-bv3n5jil
Model status        : Optimal
Objective value     :  9.5877560736e+07
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.01
Writing the solution to /private/var/folders/r9/dprdmb3d7gs81f5ggtwv0lyw0000gn/T/linopy-solve-ub3kgiyh.sol


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 233.21it/s]
INFO:linopy.io: Writing time: 0.21s


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-i1ef5nyp has 113889 rows; 52565 cols; 219009 nonzeros
Coefficient ranges:
  Matrix [2e-02, 1e+00]
  Cost   [1e-03, 2e+05]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+03]
Presolving model
61320 rows, 52564 cols, 157680 nonzeros  0s
52560 rows, 43804 cols, 140160 nonzeros  0s
Dependent equations search running on 17520 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
52560 rows, 43804 cols, 140160 nonzeros  0s
Presolve : Reductions: rows 52560(-61329); columns 43804(-8761); elements 140160(-78849)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 8760(1.0074e+06) 0s
      20894     5.6816955702e+05 Pr: 8760(7.01228e+09) 5s
      33793     2.6732945260e+07 Pr: 8761(4.17186e+08) 10s


...

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-3bkk5rdv has 131411 rows; 61326 cols; 249206 nonzeros
Coefficient ranges:
  Matrix [1e-02, 1e+00]
  Cost   [1e-03, 2e+05]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+03]
Presolving model
65235 rows, 56480 cols, 169425 nonzeros  0s
Dependent equations search running on 17520 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
56475 rows, 47720 cols, 151905 nonzeros  0s
Presolve : Reductions: rows 56475(-74936); columns 47720(-13606); elements 151905(-97301)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 8760(1.0074e+06) 0s
      40551     5.2788174934e+07 Pr: 11574(327591) 5s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 61326 primals, 131411 duals
Objective: 5.62e+07
Solver model: available
Solver message: Optimal



      53429     5.6232152715e+07 Pr: 0(0) 6s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-3bkk5rdv
Model status        : Optimal
Simplex   iterations: 53429
Objective value     :  5.6232152715e+07
P-D objective error :  1.1262228361e-14
HiGHS run time      :          6.25
Writing the solution to /private/var/folders/r9/dprdmb3d7gs81f5ggtwv0lyw0000gn/T/linopy-solve-mys0sw1_.sol


INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


Cost of running the microgrid per year: £63,398,384.28
Cost of running the microgrid with solar per year: £62,757,657.45
Cost of running with a purpose-built SMR per year: £95,877,560.74
Percentage difference in costs: -33.88%
Percentage difference in costs (with solar): -34.54%


In [16]:
import plotly.graph_objects as go

load_scenario = DATA_CENTRE_LOAD_MW
nuclear_cost = cost_nuclear_smr[load_scenario]
microgrid_cost = cost_microgrid[load_scenario]
microgrid_with_solar_cost = cost_microgrid_with_solar[load_scenario]
cost_diff_pct = cost_difference_percentages[load_scenario]

# Bar chart data
scenarios = [
    "<b>Scenario 1</b> <i>(Baseline)</i>:<br>SMR",
    "<b>Scenario 2</b> <i>(Alternative)</i>:<br>Wind+BESS+Gas Backup Microgrid",
    "<b>Scenario 3</b> <i>(With Solar)</i>:<br>Wind+BESS+Gas Backup Microgrid+Solar"
]
costs = [nuclear_cost, microgrid_cost, microgrid_with_solar_cost]
colors = ["#636EFA", "#00CC96", "#FFA07A"]

fig = go.Figure()

# Add bars with black outlines
for i, (scenario, cost, color) in enumerate(zip(scenarios, costs, colors)):
    fig.add_trace(
        go.Bar(
            x=[scenario],
            y=[cost],
            marker=dict(color=color, line=dict(color="black", width=3)),
            name=scenario,
            showlegend=False,
            width=0.5,
        )
    )

# Add downward arrow and annotation if microgrid is cheaper
fig.add_annotation(
    x=scenarios[1],
    y=microgrid_cost,
    text=f"{abs(cost_diff_pct):.1f}% lower",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor="black",
    ax=0,
    ay=-80,  # Negative value moves annotation above the bar
    font=dict(color="black", size=16),
    bgcolor="white",
    bordercolor="black",
    borderwidth=2,
)
fig.add_annotation(
    x=scenarios[2],
    y=microgrid_with_solar_cost,
    text=f"{abs(cost_difference_percentages_with_solar[load_scenario]):.1f}% lower",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor="black",
    ax=0,
    ay=-80,  # Negative value moves annotation above the bar
    font=dict(color="black", size=16),
    bgcolor="white",
    bordercolor="black",
    borderwidth=2,
)

fig.update_layout(
    title=f"Annual Cost Comparison: SMR vs Microgrid ({load} MW DC Load)",
    yaxis_title="Annual Cost (£)",
    xaxis_title="Scenario",
    template="plotly_white",
    bargap=0.5,
    yaxis=dict(tickformat=","),
)

fig.show()

## How to calculate the Renewable Profiles like they do in PyPSA EUR. 


weather cutouts can be downloaded from this zenodo repository: https://zenodo.org/records/12791128 

These files are merged spatiotemporal subsets of the European weather data from the ECMWF ERA5 realanlysis dataset and CMSAF SARAH-3 solar radiation dataset compiled by the atlite tool by various authors of PyPSA Eur 

The following script is modified from the build_renewable_profiles.py file in scripts of PyPSA-EUR. Running just soley this element of PYSPA-EUR caused issues with time alignment, thus I've replicated it here. This can also be used for any further edits of the microgrid project without fully diving into PyPSA EUR 

In [ ]:
### Documentation for Use in PyPSA-EUR 

# SPDX-FileCopyrightText: Contributors to PyPSA-Eur <https://github.com/pypsa/pypsa-eur>
#
# SPDX-License-Identifier: MIT
"""
Calculates for each clustered region the (i) installable capacity (based on
land-use from :mod:`determine_availability_matrix`), (ii) the available
generation time series (based on weather data), and (iii) the average distance
from the node for onshore wind, AC-connected offshore wind, DC-connected
offshore wind and solar PV generators.

.. note:: Hydroelectric profiles are built in script :mod:`build_hydro_profiles`.

Outputs
-------

- ``resources/profile_{technology}.nc`` with the following structure

    ===================  ==========  =========================================================
    Field                Dimensions  Description
    ===================  ==========  =========================================================
    profile              bus, time   the per unit hourly availability factors for each bus
    -------------------  ----------  ---------------------------------------------------------
    p_nom_max            bus         maximal installable capacity at the bus (in MW)
    -------------------  ----------  ---------------------------------------------------------
    average_distance     bus         average distance of units in the region to the
                                     grid bus for onshore technologies and to the shoreline
                                     for offshore technologies (in km)
    ===================  ==========  =========================================================

    - **profile**

    .. image:: img/profile_ts.png
        :scale: 33 %
        :align: center

    - **p_nom_max**

    .. image:: img/p_nom_max_hist.png
        :scale: 33 %
        :align: center

    - **average_distance**

    .. image:: img/distance_hist.png
        :scale: 33 %
        :align: center

Description
-----------

This script functions at two main spatial resolutions: the resolution of the
clustered network regions, and the resolution of the cutout grid cells for the
weather data. Typically the weather data grid is finer than the network regions,
so we have to work out the distribution of generators across the grid cells
within each region. This is done by taking account of a combination of the
available land at each grid cell (computed in
:mod:`determine_availability_matrix`) and the capacity factor there.

Based on the availability matrix, the script first computes how much of the
technology can be installed at each cutout grid cell. To compute the layout of
generators in each clustered region, the installable potential in each grid cell
is multiplied with the capacity factor at each grid cell. This is done since we
assume more generators are installed at cells with a higher capacity factor.

.. image:: img/offwinddc-gridcell.png
    :scale: 50 %
    :align: center

.. image:: img/offwindac-gridcell.png
    :scale: 50 %
    :align: center

.. image:: img/onwind-gridcell.png
    :scale: 50 %
    :align: center

.. image:: img/solar-gridcell.png
    :scale: 50 %
    :align: center

This layout is then used to compute the generation availability time series from
the weather data cutout from ``atlite``.

The maximal installable potential for the node (`p_nom_max`) is computed by
adding up the installable potentials of the individual grid cells.
"""



In [ ]:
#connection timeout with this code, but the layout should be correct.
import time

import atlite
import geopandas as gpd
import xarray as xr
#from _helpers import configure_logging, get_snapshots, set_scenario_config
#from build_shapes import _simplify_polys
from dask.distributed import Client

#from build shapes
from itertools import takewhile
from operator import attrgetter

import country_converter as coco
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from rasterio.mask import mask
from shapely.geometry import MultiPolygon, Polygon, box

#logger = logging.getLogger(__name__)

#from other pypsa EUR files like _helpers
renewable_technologies = {
    "onwind": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "wind",
            "turbine": "Vestas_V112_3MW",
            "smooth": False,
            "add_cutout_windspeed": True
        },
        "capacity_per_sqkm": 3,
        # "correction_factor": 0.93,
        "corine": {
            "grid_codes": [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32],
            "distance": 1000,
            "distance_grid_codes": [1, 2, 3, 4, 5, 6]
        },
        "luisa": False,
        # "luisa": {
        #     "grid_codes": [1111, 1121, 1122, 1123, 1130, 1210, 1221, 1222, 1230, 1241, 1242],
        #     "distance": 1000,
        #     "distance_grid_codes": [1111, 1121, 1122, 1123, 1130, 1210, 1221, 1222, 1230, 1241, 1242]
        # },
        "natura": True,
        "excluder_resolution": 100,
        "clip_p_max_pu": 1e-2
    },
    "offwind-ac": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "wind",
            "turbine": "NREL_ReferenceTurbine_2020ATB_5.5MW",
            "smooth": False,
            "add_cutout_windspeed": True
        },
        "capacity_per_sqkm": 2,
        "correction_factor": 0.8855,
        "corine": [44, 255],
        "luisa": False,  # [0, 5230]
        "natura": True,
        "ship_threshold": 400,
        "max_depth": 60,
        "max_shore_distance": 30000,
        "excluder_resolution": 200,
        "clip_p_max_pu": 1e-2,
        "landfall_length": 10
    },
    "offwind-dc": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "wind",
            "turbine": "NREL_ReferenceTurbine_2020ATB_5.5MW",
            "smooth": False,
            "add_cutout_windspeed": True
        },
        "capacity_per_sqkm": 2,
        "correction_factor": 0.8855,
        "corine": [44, 255],
        "luisa": False,  # [0, 5230]
        "natura": True,
        "ship_threshold": 400,
        "max_depth": 60,
        "min_shore_distance": 30000,
        "excluder_resolution": 200,
        "clip_p_max_pu": 1e-2,
        "landfall_length": 10
    },
    "offwind-float": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "wind",
            "turbine": "NREL_ReferenceTurbine_5MW_offshore",
            "smooth": False,
            "add_cutout_windspeed": True
        },
        "capacity_per_sqkm": 2,
        "correction_factor": 0.8855,
        "corine": [44, 255],
        "natura": True,
        "ship_threshold": 400,
        "excluder_resolution": 200,
        "min_depth": 60,
        "max_depth": 1000,
        "clip_p_max_pu": 1e-2,
        "landfall_length": 10
    },
    "solar": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "pv",
            "panel": "CSi",
            "orientation": {
                "slope": 35.0,
                "azimuth": 180.0
            }
        },
        "capacity_per_sqkm": 5.1,
        # "correction_factor": 0.854337,
        "corine": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 26, 31, 32],
        "luisa": False,  # [1111, ..., 3330]
        "natura": True,
        "excluder_resolution": 100,
        "clip_p_max_pu": 1e-2
    },
    "solar-hsat": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "pv",
            "panel": "CSi",
            "orientation": {
                "slope": 35.0,
                "azimuth": 180.0,
                "tracking": "horizontal"
            }
        },
        "capacity_per_sqkm": 4.43,
        "corine": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 26, 31, 32],
        "luisa": False,  # [1111, ..., 3330]
        "natura": True,
        "excluder_resolution": 100,
        "clip_p_max_pu": 1e-2
    },
    "hydro": {
        "cutout": "europe-2013-sarah3-era5",
        "carriers": ["ror", "PHS", "hydro"],
        "PHS_max_hours": 6,
        "hydro_max_hours": "energy_capacity_totals_by_country",  # or estimate_by_large_installations, or float
        "flatten_dispatch": False,
        "flatten_dispatch_buffer": 0.2,
        "clip_min_inflow": 1.0,
        "eia_norm_year": False,
        "eia_correct_by_capacity": False,
        "eia_approximate_missing": False
    }
}
def get_snapshots(snapshots, drop_leap_day=False, freq="h", **kwargs):
    """
    Returns pandas DateTimeIndex potentially without leap days.
    """

    time = pd.date_range(freq=freq, **snapshots, **kwargs)
    if drop_leap_day and time.is_leap_year.any():
        time = time[~((time.month == 2) & (time.day == 29))]

    return time
def _simplify_polys(
    polys, minarea=100 * 1e6, maxdistance=None, tolerance=None, filterremote=True
):  # 100*1e6 = 100 km² if CRS is DISTANCE_CRS
    if isinstance(polys, MultiPolygon):
        polys = sorted(polys.geoms, key=attrgetter("area"), reverse=True)
        mainpoly = polys[0]
        mainlength = np.sqrt(mainpoly.area / (2.0 * np.pi))

        if maxdistance is not None:
            mainlength = maxdistance

        if mainpoly.area > minarea:
            polys = MultiPolygon(
                [
                    p
                    for p in takewhile(lambda p: p.area > minarea, polys)
                    if not filterremote or (mainpoly.distance(p) < mainlength)
                ]
            )
        else:
            polys = mainpoly
    if tolerance is not None:
        polys = polys.simplify(tolerance=tolerance)
    return polys

#options from configuration file 
technology_choice = 'offwind-dc'
snakemake_threads = 4 
snakemake_snapshots = ['']
snapshots = {
  'start': "2023-01-01",
  'end': "2024-01-01",
  'inclusive': 'left',
}
cutout_filepath = '/Users/katherine.shaw/Desktop/pypsa-eur/cutouts/europe-2023-sarah3-era5.nc'
availability_matrix_filepath_dc = '/Users/katherine.shaw/Desktop/pypsa-eur/resources/availability_matrix_39_offwind-dc.nc'
snakemake_input_regions = '/Users/katherine.shaw/Desktop/pypsa-eur/resources/regions_offshore_base_s_39.geojson'

nprocesses = snakemake_threads #int(snakemake.threads)
technology = technology_choice 
params = renewable_technologies[technology]
resource = params["resource"]  # pv panel params / wind turbine params

tech = next(t for t in ["panel", "turbine"] if t in resource)
models = resource[tech]
if not isinstance(models, dict):
    models = {0: models}
resource[tech] = models[next(iter(models))]

correction_factor = params.get("correction_factor", 1.0)
capacity_per_sqkm = params["capacity_per_sqkm"]

if correction_factor != 1.0:
    print(f"correction_factor is set as {correction_factor}")

if nprocesses > 1:
    client = Client(n_workers=nprocesses, threads_per_worker=1)
else:
    client = None

sns = get_snapshots(snapshots, drop_leap_day=False)

cutout = atlite.Cutout(cutout_filepath).sel(time=sns)

availability = xr.open_dataarray(availability_matrix_filepath_dc) #this is a file made my pypsa_eur 

regions = gpd.read_file(snakemake_input_regions) #this is also a file make by pypsa year
assert not regions.empty, (
    f"List of regions in {snakemake_input_regions} is empty, please "
    "disable the corresponding renewable technology"
)
# do not pull up, set_index does not work if geo dataframe is empty
regions = regions.set_index("name").rename_axis("bus")
if technology_choice.startswith("offwind"):
    # for offshore regions, the shortest distance to the shoreline is used
    offshore_regions = availability.coords["bus"].values
    regions = regions.loc[offshore_regions]
    regions = regions.map(lambda g: _simplify_polys(g, minarea=1)).set_crs(
        regions.crs
    )
else:
    # for onshore regions, the representative point of the region is used
    regions = regions.representative_point()
    regions = regions.geometry.to_crs(3035)
    buses = regions.index

    area = cutout.grid.to_crs(3035).area / 1e6
    area = xr.DataArray(
        area.values.reshape(cutout.shape), [cutout.coords["y"], cutout.coords["x"]]
    )

    func = getattr(cutout, resource.pop("method"))
    if client is not None:
        resource["dask_kwargs"] = {"scheduler": client}

    print(f"Calculate average capacity factor for technology {technology}...")
    start = time.time()

    capacity_factor = correction_factor * func(capacity_factor=True, **resource)
    layout = capacity_factor * area * capacity_per_sqkm

    duration = time.time() - start
    print(
        f"Completed average capacity factor calculation for technology {technology} ({duration:2.2f}s)"
    )

    profiles = []
    for year, model in models.items():
        print(
            f"Calculate weighted capacity factor time series for model {model} for technology {technology}..."
        )
        start = time.time()

        resource[tech] = model

        profile = func(
            matrix=availability.stack(spatial=["y", "x"]),
            layout=layout,
            index=buses,
            per_unit=True,
            return_capacity=False,
            **resource,
        )

        dim = {"year": [year]}
        profile = profile.expand_dims(dim)

        profiles.append(profile.rename("profile"))

        duration = time.time() - start
        print(
            f"Completed weighted capacity factor time series calculation for model {model} for technology {technology} ({duration:2.2f}s)"
        )

    profiles = xr.merge(profiles)

    print(f"Calculating maximal capacity per bus for technology {technology}")
    p_nom_max = capacity_per_sqkm * availability @ area

    print(f"Calculate average distances for technology {technology}.")
    layoutmatrix = (layout * availability).stack(spatial=["y", "x"])

    coords = cutout.grid.representative_point().to_crs(3035)

    average_distance = []
    for bus in buses:
        row = layoutmatrix.sel(bus=bus).data
        nz_b = row != 0
        row = row[nz_b]
        co = coords[nz_b]
        distances = co.distance(regions[bus]).div(1e3)  # km
        average_distance.append((distances * (row / row.sum())).sum())

    average_distance = xr.DataArray(average_distance, [buses])

    ds = xr.merge(
        [
            correction_factor * profiles,
            p_nom_max.rename("p_nom_max"),
            average_distance.rename("average_distance"),
        ]
    )
    # select only buses with some capacity and minimal capacity factor
    mean_profile = ds["profile"].mean("time")
    if "year" in ds.indexes:
        mean_profile = mean_profile.max("year")

    ds = ds.sel(
        bus=(
            (mean_profile > params.get("min_p_max_pu", 0.0))
            & (ds["p_nom_max"] > params.get("min_p_nom_max", 0.0))
        )
    )

    if "clip_p_max_pu" in params:
        min_p_max_pu = params["clip_p_max_pu"]
        ds["profile"] = ds["profile"].where(ds["profile"] >= min_p_max_pu, 0)

    
    if client is not None:
        client.shutdown()


## Comparison of 2019 and 2023 

In [ ]:
#wind and solar availability update 
##renewable profile for 2023
#the 2023 file from the respoitory above is for the year 2023
cutout_2023 = xr.load_dataset('/Users/katherine.shaw/Desktop/pypsa-eur/cutouts/europe-2023-sarah3-era5.nc') 
#cutout defined from pypsa eur dataset
#cutout can be downloaded from this zenedo repository : https://zenodo.org/records/12791128


#2019 
#wind data 
wind_data = xr.load_dataset(
    "/Users/katherine.shaw/Desktop/pypsa-eur/resources/profile_39_offwind-dc.nc",
)
wind_profile_df = wind_data["profile"].to_dataframe().reset_index()
wind_availability_list = wind_profile_df[wind_profile_df["bus"] == "GB2 0"]["profile"].to_numpy()

#solar data 
solar_profile = xr.load_dataset('/Users/katherine.shaw/Desktop/pypsa-eur/resources/profile_39_solar.nc')
solar_availability_df = solar_profile['profile'].to_dataframe().reset_index()
solar_availability_df = solar_availability_df[solar_availability_df.bus == 'GB2 0']
solar_availability_df.reset_index()
solar_availability_df = solar_availability_df.profile
solar_availability = solar_availability_df.to_numpy()

#solar capital cost
solar_capex_per_mw = (700.13 * 1000) # originally given in EUR/kW_e, so had to multiply by 1000 to make /MW and also turn into £ to match wind cost units.  taken from pypsa eur cost files 
solar_discount_rate = wind_discount_rate
solar_lifetime_years = 35.0 #taken from pypsa eur cost files 
annuitised_solar_cost = (solar_capex_per_mw * solar_discount_rate) / (
    1 - (1 + solar_discount_rate) ** -solar_lifetime_years
)



In [ ]:
# Convert your data to a pandas Series with datetime index
fig, ax = plt.subplots(figsize = (14,6)) 

wind_series = pd.Series(wind_availability_list, index=snapshots)
wind_series2023 = pd.Series(wind_profile_df2023, index = snapshots)
# Apply 3-day rolling mean
# Assuming hourly data: 3 days * 24 hours = 72
rolling_mean = wind_series.rolling(window=72, center=True).mean()
rolling_mean2023 = wind_series2023.rolling(window=72, center=True).mean()
# Plot
plt.plot(wind_series.index, rolling_mean, color='darkblue', label = '2019')
plt.plot(wind_series.index, rolling_mean2023, color='lightblue', label = '2023')
plt.title('3-Day Rolling Mean of Wind Availability')
plt.ylabel('Wind Availability')
plt.xlabel('Time')
plt.legend()
plt.show()

In [ ]:

#plt.plot(snapshots, list(solar_availability), color = 'darkred')
#plt.plot(snapshots, list(solar_availability2023), color = 'lightcoral')


# Convert your data to a pandas Series with datetime index
fig, ax = plt.subplots(figsize = (14,6)) 

solar_series = pd.Series(solar_availability, index=snapshots)
solar_series2023 = pd.Series(solar_availability2023, index = snapshots)
# Apply 3-day rolling mean
# Assuming hourly data: 3 days * 24 hours = 72
rolling_mean = solar_series.rolling(window=72, center=True).mean()
rolling_mean2023 = solar_series2023.rolling(window=72, center=True).mean()
# Plot
plt.plot(wind_series.index, rolling_mean, color = 'darkred', label = '2019')
plt.plot(wind_series.index, rolling_mean2023, color = 'lightcoral', label = '2023')
plt.title('3-Day Rolling Mean of Solar Availability')
plt.ylabel('Solar Availability')
plt.xlabel('Time')
plt.legend()
plt.show()

In [ ]:
solar_availability.sum()
solar_availability2023.sum()